# Bedrock onboarding notebook

This notebook provides steps requried to install Bedrock SDK, dependencies and other pre-requisistes before API calls can be made.
It also provides sample code to access LLMs, text to image, embeddings and streaming support

(This notebook was tested on SageMaker Studio ml.m5.2xlarge instance with Datascience 3.0 kernel)


Before you proceed further, ensure following steps are carried out: 

* Ensure that you have access to Amazon Bedrock service in your account and the region you are launching these notebooks
* Review Amazon Bedrock service user guide. You can access the user guide [here.](https://d2eo22ngex1n9g.cloudfront.net/Documentation/BedrockUserGuide.pdf)
* Subscribe to the models you plan to utilize. This requires subscription to the models from AWS Marketplace for third-party models. If your account is setup with Private Marketplace, you need to subscribe to the model Ids and they need to be part of the Private Marketplace products. Refer to Bedrock User guide for details.
* Fill-out forms on Bedrock servie console to get access to the models. The access forms can be filled-in from Playground page on Amazon Bedrock console page. The subscription requests take few days and you would see a message "Request Pending" in Playground page until the approval is received.
* Setup IAM policies to get access to Amazon Bedrock service. You can refer to Bedrock User Guide for a list of actions.

## Pre-requisites

In [ ]:
#Check Python version is greater than 3.8 which is required by Langchain if you want to use Langchain
import sys
sys.version

In [ ]:
assert sys.version_info >= (3, 8)

## Step 1 - Copy the documentation/SDK folder

In [ ]:
!wget https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk.zip -P bedrock_docs/

## Step 2 - Install the SDK

#### Unzip the SDK - this may take some time

In [ ]:
!unzip -o bedrock_docs/bedrock-python-sdk.zip -d bedrock_docs/SDK-1-28

#### Uninstall previous version of SDK

In [ ]:
#Change the directory if previous version is in a different location
!python3 -m pip uninstall bedrock_docs/SDK/boto3-1.26.162-py3-none-any.whl -y

In [ ]:
!python3 -m pip uninstall bedrock_docs/SDK/botocore-1.29.162-py3-none-any.whl -y

### Install latest SDK

Install the boto3 package. This overwrites any existing boto3 installations and will not break any other calls to other services. Also, if you are running this in a notebook, you may need to restart the kernel after installation 

In [ ]:
#!pip uninstall sagemaker -y # results in install errors with latest sagemaker, and can be installed later

In [ ]:
!python3 -m pip install bedrock_docs/SDK-1-28/boto3-1.28.21-py3-none-any.whl

In [ ]:
!python3 -m pip install bedrock_docs/SDK-1-28/botocore-1.31.21-py3-none-any.whl

In [ ]:
 # Optional, for the CLI:
#!python3 -m pip install bedrock_docs/SDK-1-28/awscli-1.29.21-py3-none-any.whl

### Verify boto3 installation

In [ ]:
#!aws bedrock list-foundation-models 
!aws --version

In [ ]:
#Check if boto3 is installed & get the install location
!pip show boto3

In [ ]:
!ls /opt/conda/lib/python3.10/site-packages/botocore/data | grep bedrock

## Restart Kernel

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)  

## Step 3- Attach policies to IAM role to permission Bedrock service

In [ ]:
import sagemaker
import boto3
session = boto3.Session()
sagemaker_session = sagemaker.Session()
studio_region = sagemaker_session.boto_region_name 
#sagemaker_session.get_caller_identity_arn()

First add a policy to the role listed above similar to:
    
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "Bedrock",
            "Effect": "Allow",
            "Action": "bedrock:*",
            "Resource": "*"
        }
    ]
}
```

## Step 4 - Test bedrock boto3 install

In [ ]:
# Configure your AWS credentials using the aws configure command, or pass them to the boto3 client
bedrock = boto3.client('bedrock' , 'us-east-1', endpoint_url='https://bedrock.us-east-1.amazonaws.com')
bedrock.list_foundation_models()

In [ ]:
[m['modelId'] for m in bedrock.list_foundation_models()['modelSummaries']]

## Step 5- Test Foundation models 

In [ ]:
import json
prompt_data = """Command: Write me a blog about making strong business decisions as a leader.\nBlog:"""

### Evaluate Titan Large

In [ ]:
body = json.dumps({"inputText": prompt_data})
modelId = "amazon.titan-tg1-large"  
accept = "application/json"
contentType = "application/json"

response = bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("results")[0].get("outputText"))

### Evaluate Anthropic Claude Instant

In [ ]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  
accept = "application/json"
contentType = "application/json"

response = bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completion"))

### Evaluate AI21 Jurassic Grande

In [ ]:
body = json.dumps({"prompt": prompt_data, "maxTokens": 200})
modelId = "ai21.j2-grande-instruct"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completions")[0].get("data").get("text"))

### Evaluate Titan Embeddings

In [ ]:
body = json.dumps({"inputText": prompt_data})
modelId = "amazon.titan-e1t-medium"  
accept = "application/json"
contentType = "application/json"

response = bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

embedding = response_body.get("embedding")
print(f"The embedding vector has {len(embedding)} values\n{embedding[0:3]+['...']+embedding[-3:]}")

### Evaluate Stable Diffusion Model

In [ ]:
prompt_data = "a fine image of an astronaut riding a horse on Mars"
body = json.dumps({
  "text_prompts": [
    { 
      "text": prompt_data 
    }
  ],
  "cfg_scale":10,
  "seed":20,
  "steps":50
})
modelId = "stability.stable-diffusion-xl" 
accept = "application/json"
contentType = "application/json"

response = bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body['result'])
print(f'{response_body.get("artifacts")[0].get("base64")[0:80]}...')

In [ ]:
import io, base64
from matplotlib.pyplot import imshow
from PIL import Image
base_64_img_str = response_body.get("artifacts")[0].get("base64")
image = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
imshow(image)